# **BIBLIOTECAS**

In [1]:
import pandas as pd
import numpy as np
import requests
import time
import datetime
import re
import string
import matplotlib.pyplot as plt
import nltk
from textblob import TextBlob
from deep_translator import GoogleTranslator
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup

# **FUNÇÕES**

## **UTILIDADES**

### **CRIANDO DATASET**

In [23]:
def create_dataset():
  return pd.DataFrame(columns = ['Data', 'endereco'])

### **LISTANDO DIAS**

In [24]:
def all_days_since(since: str):

    """
    Creates a list of dates with all days since a given date.
    The start_date parameter must be a string date with the format: '%Y-%m-%d'
    """

    return pd.date_range(start=since, end=datetime.datetime.today().strftime('%Y-%m-%d'))

## **WEB SCRAPING**

### **REQUEST G1**

In [25]:
def get_request_page(q: str, date: str, source: str):

    """
    Solicita a URL de pesquisa da fonte especificada (por exemplo, 'g1' ou 'valor econômico') 
    e retorna o objeto de solicitação se o status_code for igual a 200.
    """

    url = ""

    if source.lower() == "g1":
        url = f'https://g1.globo.com/busca/?q={q}&page=1&order=relevant&species=notícias&from={date}T00%3A00%3A00-0300&to={date}T23%3A59%3A59-0300'

    elif source.lower() == "valor":
        url = f'https://valor.globo.com/busca/?q={q}&page=1&order=relevant&species=notícias&from={date}T00%3A00%3A00-0300&to={date}T23%3A59%3A59-0300'
    
    else:
        print(f"Source '{source}' não reconhecido.")
        return False

    try:
        req = requests.get(url, timeout= 60.0)

        if req.status_code == 200:
            return req

    except Exception as e:
        err_name = type(e).__name__
        print(f'Requisition error for URL {url}: {err_name}')
        return False

### **CARREGANDO PÁGINA**

In [26]:
def get_page(response):
  soup = BeautifulSoup(response.text, 'html.parser')
  return soup.select('.widget--info__text-container a')

### **BUSCANDO LINK NOTÍCIAS G1**

In [27]:
def initialize_g1(start: str, source: str):

  dataset = create_dataset()
  days = all_days_since(start)

  for i, day in enumerate(days):
    dataset.to_excel('dataset_agro bkp.xlsx', encoding='latin-1', index=False) if i % 7 == 0 else None
    print(day)

    try:
      response = get_request_page(q = 'Agronegócio', date = str(day.date()), source = source)
      page = get_page(response = response)

      for http in page:
        
        try:
          http = http.get('href')
          dataset.loc[len(dataset)] = [day,  http]

        except Exception as e1:
          err_name = type(e1).__name__
          print(f'Nãoo achou o link {http}: {err_name}')
          continue

    except Exception as e2:
        err_name = type(e2).__name__
        print(f'Requisition error for URL {http}: {err_name}')
    
  dataset['endereco'] = "https:" + dataset['endereco']
  dataset.to_excel('dataset_agro.xlsx')
    
  return dataset

### **BUSCANDO TEXTO DAS NOTICICAS**

In [28]:
def get_http_text(dataset):
  textos_completos = []
  titulos = []

  https = dataset['endereco'].to_list()
  restantes = 0
  
  for i, http in enumerate(https):
    print(f"Restam {len(https)-restantes} consultas")
    sessao = requests.Session()
    
    try:
      url = f'{http}'
      req = sessao.get(url, timeout= 15.0)
      soup = BeautifulSoup(req.text, 'html.parser')
      body_element = soup.body
      paragrafos = [p.get_text(separator='\n', strip=True) for p in body_element.find_all('p')]
      texto_completo = '\n'.join(paragrafos)

      textos_completos.append(texto_completo)
      pd.DataFrame(textos_completos).to_excel('textos_completos_agro.xlsx', encoding='latin-1', index=False) if i % 100 == 0 else None
      restantes += 1
      sessao.close()

    except TimeoutError as e1:
       texto_completo = "N/D"
       textos_completos.append(texto_completo)
       pd.DataFrame(textos_completos).to_excel('textos_completos_agro.xlsx', encoding='latin-1', index=False) if i % 100 == 0 else None
       print(f"Erro {e1}, aguardando {time.sleep(600)} segundos")
       sessao.close()
       continue
  
    #EM CASO DE REDIRECIONAMENTO DE LINK
    except AttributeError as e2:
        
        try:
            script_redirecionamento = soup.find('script', string=re.compile(r'window\.location\.replace\("(.+)"\)'))
            match = re.search(r'window\.location\.replace\("(.+)"\)', str(script_redirecionamento))
            url_redirecionada = match.group(1)

            print(f'{e2} redirecionado para {url_redirecionada}')
            
            req = requests.get(url_redirecionada, timeout= 300.0)
            soup = BeautifulSoup(req.text, 'html.parser')
            body_element = soup.body

            paragrafos = [p.get_text(separator='\n', strip=True) for p in body_element.find_all('p')]
            texto_completo = '\n'.join(paragrafos)
            
            textos_completos.append(texto_completo)
            pd.DataFrame(textos_completos).to_excel('textos_completos_agro.xlsx', encoding='latin-1', index=False) if i % 100 == 0 else None
            restantes += 1
            sessao.close()

        except TimeoutError as e3:
            texto_completo = "N/D"
            textos_completos.append(texto_completo)
            pd.DataFrame(textos_completos).to_excel('textos_completos_agro.xlsx', encoding='latin-1', index=False) if i % 100 == 0 else None
            sessao.close()
            print(f"Erro {e3}, aguardando {time.sleep(600)} segundos")
            continue
            
        except Exception as e4:
            texto_completo = "N/D"
            textos_completos.append(texto_completo)
            pd.DataFrame(textos_completos).to_excel('textos_completos_agro.xlsx', encoding='latin-1', index=False) if i % 100 == 0 else None
            print(f"Erro ao obter URL redirecionada {url_redirecionada}: {e4}")
            time.sleep(600)
            restantes +=1
            sessao.close()
            continue  # Continua para a próxima iteração em caso de erro na URL redirecionada 

    except Exception as e:
      texto_completo = "N/D"
      textos_completos.append(texto_completo)
      pd.DataFrame(textos_completos).to_excel('textos_completos_agro.xlsx', encoding='latin-1', index=False) if i % 100 == 0 else None
      print(f"Erro ao obter URL redirecionada: {e}")
      time.sleep(600)
      restantes +=1
      sessao.close()
      continue

  dataset['Texto'] = textos_completos
  dataset.to_excel('textos_completos_agro.xlsx', index = False)

  return dataset

## **TRATAMENTO DOS DADOS**

### **REMOVENDO NOTÍCIAS DUPLICADAS**

In [4]:
def drop_duplicate_news(dataset):
    dataset= dataset.groupby('Texto', as_index=False).first()
    dataset = dataset.sort_values(by=['Data'])
    dataset = dataset[['Data', 'endereco', 'Texto', 'Num_Palavras-Chave']]
    return dataset

### **REMOVENDO NOTICIAS NÃO RELEVANTES**

In [5]:
def remove_useless(dataset, min_crit=1):

    palavras_chave = [
        'Agronegócio',
        'Agricultura',
        'Pecuária',
        'Agroindústria',
        'Produção Agrícola',
        'Cultivo',
        'Plantio',
        'Colheita',
        'Agropecuária',
        'Gestão Rural',
        'Tecnologia no Campo',
        'Irrigação',
        'Fertilizantes',
        'Defensivos Agrícolas',
        'Mercado Agrícola',
        'Exportação Agrícola',
        'Logística Agrícola',
        'Sustentabilidade no Agronegócio',
        'Agrobusiness',
        'Cadeia Produtiva'
    ]
      
    dataset['Texto'] = dataset['Texto'].astype(str)
    
    # Função para contar o número de palavras-chave encontradas em cada texto
    def count_keywords(text):
        return sum(keyword.lower() in text.lower() for keyword in palavras_chave)
    
    # Criar uma nova coluna com o número de palavras-chave encontradas para cada texto
    dataset['Num_Palavras-Chave'] = dataset['Texto'].apply(count_keywords)
    
    # Aplicar a máscara para manter apenas os textos relevantes
    mask = dataset['Num_Palavras-Chave'] >= min_crit
    dataset = dataset[mask]
    
    dataset.reset_index(drop=True, inplace=True)
    
    return dataset

### **REMOVENDO STOPWORDS**

In [17]:
def remove_stopwords_br(text):
    stop_words_pt = set(stopwords.words('portuguese'))    
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words_pt]
    return ' '.join(filtered_tokens)

def remove_stopwords_en(text):
    stop_words_pt = set(stopwords.words('english'))    
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words_pt]
    return ' '.join(filtered_tokens)

### **REMOVENDO POTUAÇÃO**

In [18]:
def remove_pontuaiton(text):
    punctuation = set(string.punctuation)
    for i in text:
        if i in punctuation:
            text = text.replace(i, "")
            
    return text

### **REMOVENDO NÚMEROS**

In [19]:
def remove_numbers(text):
    string = text
    return ''.join(filter(lambda z: not z.isdigit(), string))

### **REMOVENDO EXPRESSOES REGULARES**

In [6]:
#def remove_expressoes(text):
#    return re.sub(r"\\n|\\r|-", " ", text)

def remove_expressoes(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r'\r', " ", text)
    text = re.sub(r'-', ' ', text)
    text = re.sub(r'\t', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    return text

In [8]:
def tradutor(texto):
    #time.sleep()  # Adiciona um atraso de 1 segundo entre chamadas
    print('traduzindo')
    return TextBlob(texto).translate(from_lang="pt", to='en')

### **TRATANDO OS DADOS**

In [21]:
def tratar_dados(dataset):
    dataset = remove_useless(dataset)
    dataset.loc[:, 'Texto'] = dataset['Texto'].map(remove_expressoes)
    dataset['Traducao'] = dataset['Texto'].apply(tradutor)
    
    dataset['Texto'] = dataset['Texto'].map(lambda x:str(x).lower())
    dataset[['Traducao', 'Texto']] = dataset[['Traducao', 'Texto']].applymap(remove_stopwords_br)
    dataset[['Traducao', 'Texto']] = dataset[['Traducao', 'Texto']].applymap(remove_stopwords_en)
    dataset[['Traducao', 'Texto']] = dataset[['Traducao', 'Texto']].applymap(remove_pontuaiton)
    dataset[['Traducao', 'Texto']] = dataset[['Traducao', 'Texto']].applymap(remove_numbers)

    dataset.to_excel('noticicas_tratadas_agronegocio.xlsx', index = False)

    return dataset

def tratar_dados_temp(dataset):
    dataset = remove_useless(dataset)
    dataset = drop_duplicate_news(dataset)
    
    dataset.loc[:, 'Texto'] = dataset['Texto'].map(remove_expressoes)

    print("Iniciando Traduções")
    dataset['Traducao'] = dataset['Texto'].apply(lambda x: TextBlob(x).translate(from_lang="pt", to='en')).astype('str')

    dataset['Texto'] = dataset['Texto'].map(lambda x:str(x).lower())
    dataset['Texto'] = dataset['Texto'].map(remove_stopwords_br)
    dataset['Texto'] = dataset['Texto'].map(remove_stopwords_en)
    dataset['Texto'] = dataset['Texto'].map(remove_pontuaiton)
    dataset['Texto'] = dataset['Texto'].map(remove_numbers)

    dataset.to_excel('noticicas_tratadas_agronegocio.xlsx', index = False)

    return dataset

## **ANÁLISE DE SENTIMENTOS**

### **TOKENIZAÇÃO**

In [22]:
def tokenize_text(text):
    if isinstance(text, str):
        return word_tokenize(text)
    else:
        return []

def token_br(dataset_noticias):
    # Aplicar a tokenização à coluna 'texto'
    dataset_noticias['token'] = dataset_noticias['Texto'].apply(tokenize_text)

    # Replicar as linhas para cada palavra
    dataset_token = dataset_noticias.explode('token')
    dataset_token.drop(['Texto', 'endereco', 'Num_Palavras-Chave'], axis=1, inplace=True)
    dataset_token.reset_index(drop=True, inplace=True)
    

    return dataset_token

def token_en(dataset_noticias):
    # Aplicar a tokenização à coluna 'texto'
    dataset_noticias['token'] = dataset_noticias['Traducao'].apply(tokenize_text)

    # Replicar as linhas para cada palavra
    dataset_token = dataset_noticias.explode('token')
    dataset_token.drop(['Texto'], axis=1)
    dataset_token.reset_index(drop=True, inplace=True)
    dataset_token = dataset_token[['Data', 'token']]

    return dataset_token

### **ANALISE DE SENTIMENTOS**

In [23]:
def sentiment_analysis_br(dataset_token):
    Loughan_Mc = pd.read_excel(r"C:\Users\eusou\OneDrive\Documentos\TCC\06. Dicionários\Loughran_McDonald_pt.xlsx")

    Loughan_Mc = Loughan_Mc.loc[(Loughan_Mc['sentimento'] == 'positivo') | (Loughan_Mc['sentimento'] == 'negativo')]
    
    sentiment_analysis = None
    sentiment_analysis = pd.merge(dataset_token, Loughan_Mc, on='token')
    sentiment_analysis['Data'] = pd.to_datetime(sentiment_analysis['Data'], format='%d/%m/%Y')
    sentiment_analysis = sentiment_analysis.groupby(['Data', 'sentimento']).size().unstack(fill_value=0)
    sentiment_analysis['Sentimento'] = ((sentiment_analysis['positivo'] - sentiment_analysis['negativo']) / (sentiment_analysis['positivo'] + sentiment_analysis['negativo']))

    sentiment_analysis.to_excel('Sentiment Analysis_agrongocio_br.xlsx')

    return sentiment_analysis

def sentiment_analysis_en(dataset_token):
    Loughan_Mc = pd.read_excel(r"C:\Users\eusou\OneDrive\Documentos\TCC\06. Dicionários\Loughran_McDonald_en.xlsx")
    Loughan_Mc = Loughan_Mc.loc[(Loughan_Mc['sentiment'] == 'positive') | (Loughan_Mc['sentiment'] == 'negative')]
    
    sentiment_analysis = None
    sentiment_analysis = pd.merge(dataset_token, Loughan_Mc, on='token')
    sentiment_analysis['Data'] = pd.to_datetime(sentiment_analysis['Data'], format='%d/%m/%Y')
    sentiment_analysis = sentiment_analysis.groupby(['Data', 'sentiment']).size().unstack(fill_value=0)
    sentiment_analysis['sentiment'] = ((sentiment_analysis['positive'] - sentiment_analysis['negative']) / (sentiment_analysis['positive'] + sentiment_analysis['negative']))

    sentiment_analysis.to_excel('Sentiment Analysis_mercado_agronegocio_en.xlsx')

    return sentiment_analysis

### **SEMENALIZANDO OS SENTIMENTOS**

In [24]:
def semanalização(sentiment_analysis):

    sentiment_analysis = sentiment_analysis.resample("M").mean()

    return sentiment_analysis

### **PLOT TS**

In [25]:
def plot_sentiment_analysis(sentiment_analysis, title:str):

    plt.figure(figsize=(12, 6))
    plt.plot(sentiment_analysis.index, sentiment_analysis['Sentimento'], label='Sentimento', linestyle='-')

    plt.title(title)
    plt.xlabel('Data')
    plt.ylabel('Contagem')
    plt.legend()
    plt.grid(True)
    return plt.show()

# **LABORATÓRIO**

In [ ]:
dataset = initialize_g1(start='2010-05-01', source= 'g1')
dataset_text = get_http_text(dataset)

In [74]:
# APENAS UM CRITÉRIO POIS NÃO HÁ TANTAS IRREGULARIDADES NA BASE DE DADOS

dataset_text = pd.read_excel('2.textos_completos_agro.xlsx')
noticias_tratadas = tratar_dados_temp(dataset_text)
tokens_br = token_br(noticias_tratadas)
sentiment_analysis_br = sentiment_analysis_br(tokens_br)

### TRADUÇÃO

In [ ]:
def tratar_dados_temp(dataset):
    dataset = remove_useless(dataset)
    dataset = drop_duplicate_news(dataset)
    
    dataset.loc[:, 'Texto'] = dataset['Texto'].map(remove_expressoes)

    print("Iniciando Traduções")
    dataset['Traducao'] = dataset['Texto'].apply(lambda x: TextBlob(x).translate(from_lang="pt", to='en')).astype('str')

    dataset['Texto'] = dataset['Texto'].map(lambda x:str(x).lower())
    dataset['Texto'] = dataset['Texto'].map(remove_stopwords_br)
    dataset['Texto'] = dataset['Texto'].map(remove_stopwords_en)
    dataset['Texto'] = dataset['Texto'].map(remove_pontuaiton)
    dataset['Texto'] = dataset['Texto'].map(remove_numbers)

    dataset.to_excel('noticicas_tratadas_agronegocio.xlsx', index = False)

    return dataset

In [42]:
#noticias_tratadas = tratar_dados_temp(y)

In [3]:
#y['Texto'] = y['Texto']
#y['Texto'] = y['Texto'].map(lambda x:Word(x).maketrans())
y = pd.read_excel('2.textos_completos_agro.xlsx')

In [7]:
y = remove_useless(y)
y = drop_duplicate_news(y)
y.loc[:, 'Texto'] = y['Texto'].map(remove_expressoes)
y.loc[:, 'Texto'] = y['Texto'].map(lambda x:str(x).replace("  ", " "))

In [8]:
y = y.iloc[:100]

In [9]:
def translate(texto):

    tradutor = GoogleTranslator(source="pt", target="en")

    try:
        texto_traduzido = tradutor.translate(texto)
        return texto_traduzido
    
    except:
        print("Texto excede os 5mil caracteres")
        return "Texto excede os 5mil caracteres"

y['Tradução'] = y['Texto'].map(translate)

#12m

Texto excede os 5mil caracteres
Texto excede os 5mil caracteres
Texto excede os 5mil caracteres
Texto excede os 5mil caracteres
Texto excede os 5mil caracteres
Texto excede os 5mil caracteres
Texto excede os 5mil caracteres
Texto excede os 5mil caracteres
Texto excede os 5mil caracteres
Texto excede os 5mil caracteres
Texto excede os 5mil caracteres
Texto excede os 5mil caracteres
Texto excede os 5mil caracteres
Texto excede os 5mil caracteres
